In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

client = OpenAI()

# Single review compression

In [3]:
# Single review

prompt = """
Give sentiment and themes for the given input review.

Give output as json with "sentiment" key and value should be either of [POSITIVE, NEUTRAL, NEGATIVE]. 
And another key "themes" with list of themes talked about in the review, for example "Producut Issue", "Packaging Issue", etc.

Strictly only give json output as string and nothing else. Do not give markdown.

Input Review:
```
{input_review}
```
"""

In [4]:
input_review = "This bedsheet is beautiful and looks just like in the picture. The print is very pretty. It is really comfortable and smooth. Fabric is very nice , color doesn’t fade even after the wash. It fits our queen size bed perfectly ( it can be easy used for King size bed) . Go for it, value for money."

formatted_prompt = prompt.format(input_review=input_review)
print(formatted_prompt)


Give sentiment and themes for the given input review.

Give output as json with "sentiment" key and value should be either of [POSITIVE, NEUTRAL, NEGATIVE]. 
And another key "themes" with list of themes talked about in the review, for example "Producut Issue", "Packaging Issue", etc.

Strictly only give json output as string and nothing else. Do not give markdown.

Input Review:
```
This bedsheet is beautiful and looks just like in the picture. The print is very pretty. It is really comfortable and smooth. Fabric is very nice , color doesn’t fade even after the wash. It fits our queen size bed perfectly ( it can be easy used for King size bed) . Go for it, value for money.
```



In [5]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": formatted_prompt},
    ],
)

compressed_review = completion.choices[0].message.content

In [6]:
print(compressed_review)

{
  "sentiment": "POSITIVE",
  "themes": ["Product Quality", "Comfort", "Value for Money", "Design"]
}


In [7]:
d = json.loads(compressed_review)

In [8]:
d["sentiment"]

'POSITIVE'

# Multiple review compression

In [9]:
review_analysis_prompt = """
Give sentiment and themes for the given input review.

Give output as json with "sentiment" key and value should be either of [POSITIVE, NEUTRAL, NEGATIVE]. 
And another key "themes" with list of themes talked about in the review, for example "Producut Issue", "Packaging Issue", etc.

Strictly only give json output as string and nothing else. Do not give markdown.

Input Review:
```
{input_review}
```
"""


def get_review_analysis(reviews):
    review_analysis_results = []
    for review in reviews:
        prompt = review_analysis_prompt.format(input_review=review)
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
            ],
        )

        review_analysis = completion.choices[0].message.content
        print(review_analysis)
        parsed_review_analysis = json.loads(review_analysis)
        review_analysis_results.append(parsed_review_analysis)
    return review_analysis_results

In [10]:
reviews = ["This bedsheet is beautiful and looks just like in the picture. The print is very pretty. It is really comfortable and smooth. Fabric is very nice , color doesn’t fade even after the wash. It fits our queen size bed perfectly ( it can be easy used for King size bed) . Go for it, value for money.",
           "Quality is good. But it is not a king size one. I'd say it is queen size.",
           "Superb bedsheet.. soft material.. I like it . Go for it .. colour fast febric nd it looked so beautiful 😻 finally got what I need . Will order more from this brand.. happy shopping 😀🛍️",
           "In the first wash, the bedsheet faded and shrunk a lot. Don't buy this product.",
           "Product is ok for discounted price. Colour is not that bright as shown in pic...some slight defects on bedsheet....ok ok product"
          ]

In [11]:
compressed_review_result = get_review_analysis(reviews)

{"sentiment":"POSITIVE","themes":["Product Quality","Comfort","Value for Money","Design","Fit"]}
{"sentiment":"NEUTRAL","themes":["Quality","Size Issue"]}
{"sentiment":"POSITIVE","themes":["Product Quality","Material","Aesthetic Appeal","Brand Loyalty"]}
{"sentiment":"NEGATIVE","themes":["Product Issue","Quality Issue"]}
{"sentiment":"NEUTRAL","themes":["Product Quality","Discounted Price","Color Accuracy","Defects"]}


In [12]:
compressed_review_result

[{'sentiment': 'POSITIVE',
  'themes': ['Product Quality',
   'Comfort',
   'Value for Money',
   'Design',
   'Fit']},
 {'sentiment': 'NEUTRAL', 'themes': ['Quality', 'Size Issue']},
 {'sentiment': 'POSITIVE',
  'themes': ['Product Quality',
   'Material',
   'Aesthetic Appeal',
   'Brand Loyalty']},
 {'sentiment': 'NEGATIVE', 'themes': ['Product Issue', 'Quality Issue']},
 {'sentiment': 'NEUTRAL',
  'themes': ['Product Quality',
   'Discounted Price',
   'Color Accuracy',
   'Defects']}]

# Actionable insights from compressed reviews

In [13]:
actionable_insights_prompt = """I have sentiments around some themes extracted from reviews.
I want you to give me actionable insights in bullet points.
Keep it exceptionally very shrot and crisp.

Input:
```
{review_analysis}
```"""

In [14]:
prompt = actionable_insights_prompt.format(review_analysis=str(compressed_review_result))
print(prompt)

I have sentiments around some themes extracted from reviews.
I want you to give me actionable insights in bullet points.
Keep it exceptionally very shrot and crisp.

Input:
```
[{'sentiment': 'POSITIVE', 'themes': ['Product Quality', 'Comfort', 'Value for Money', 'Design', 'Fit']}, {'sentiment': 'NEUTRAL', 'themes': ['Quality', 'Size Issue']}, {'sentiment': 'POSITIVE', 'themes': ['Product Quality', 'Material', 'Aesthetic Appeal', 'Brand Loyalty']}, {'sentiment': 'NEGATIVE', 'themes': ['Product Issue', 'Quality Issue']}, {'sentiment': 'NEUTRAL', 'themes': ['Product Quality', 'Discounted Price', 'Color Accuracy', 'Defects']}]
```


In [15]:
completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
            ],
        )

actionable_insights = completion.choices[0].message.content
print(actionable_insights)

- **Strengthen Positive Aspects**: Continue to highlight product quality, comfort, and design in marketing efforts.
- **Address Size Issues**: Identify and correct any size discrepancies to reduce neutral sentiments.
- **Focus on Material Quality**: Ensure high standards for materials to maintain both aesthetic appeal and brand loyalty.
- **Resolve Product Issues**: Investigate reported product and quality issues to prevent negative experiences and enhance customer satisfaction.
- **Monitor Defects**: Keep an eye on defect rates and color accuracy to improve overall quality perception.
